## Emulation Queries

Requires the following to be performed on the DB:

```
CREATE EXTENSION tablefunc;
```

The following demonstrates how to generate a dynamic parameter table for a given model where each `run_id` has a column for its various parameters.

In [1]:
from sqlalchemy.sql import text

import sys
sys.path.append("../db")

from database import init_db, db_session, engine
from models import Metadata, Output, Parameters

import pandas as pd

postgres


Select the model of interest:

In [2]:
model = 'yield_anomalies_lpjml'

Generate that returns a string which dynamically builds the columns resulting from the crosstab:

In [3]:
col_query = f"""SELECT CONCAT('run_id VARCHAR, ', params, ' VARCHAR') 
               FROM (SELECT string_agg(DISTINCT parameter_name::text, ' VARCHAR, ') 
               AS params FROM parameters where model = '{model}') AS ps"""
print(col_query)
conn = engine.connect()
rs = conn.execute(col_query)
dynamic_cols = rs.first()[0]
print("\nDynamic column list to inject into query:")
print(dynamic_cols)

SELECT CONCAT('run_id VARCHAR, ', params, ' VARCHAR') 
               FROM (SELECT string_agg(DISTINCT parameter_name::text, ' VARCHAR, ') 
               AS params FROM parameters where model = 'yield_anomalies_lpjml') AS ps

Dynamic column list to inject into query:
run_id VARCHAR, crop VARCHAR, irrigation VARCHAR, nitrogen VARCHAR, stat VARCHAR


Generate dynamic cross tab query:

In [4]:
ct = f"""SELECT
  *
FROM
  crosstab (
     $$SELECT DISTINCT run_id, parameter_name, parameter_value FROM parameters 
     WHERE model = '{model}'
     GROUP BY run_id, parameter_name, parameter_value
     ORDER BY 1$$,
    $$SELECT DISTINCT parameter_name FROM parameters WHERE model = '{model}' ORDER BY 1$$
  ) AS (
    {dynamic_cols}
) ORDER BY run_id
"""

Read into pandas DataFrame:

In [5]:
df = pd.read_sql_query(ct, conn)

In [6]:
df.head()

,run_id,crop,irrigation,nitrogen,stat
0,03aa57b2ae8a1a9ba087d7d1266f90db8c8dc62f842e05...,wheat,NO,LIM,"pctl,95"
1,1098aef84395584d99fed198da4c167590c003db05622b...,wheat,LIM,LIM_p25,"pctl,5"
2,25afdf9b7d25dcb14f7ec3d082a1a326f651219160f03d...,wheat,LIM,LIM_p50,mean
3,2a5717269b3758392bca2eb209b5efe78e34f5e49a3ced...,wheat,NO,LIM_p25,"pctl,5"
4,3a211b54c4c8a67c7c4a1094a03a12de4b7760e77b0cd2...,wheat,NO,LIM_p25,"pctl,95"


### Querying for an "emulation"
Now we can query for "emulation" where **ONLY** nitrogen is varied:

In [7]:
emulation = f"""SELECT run_id, crop, irrigation, nitrogen, stat
                FROM ({ct}) as ct
                WHERE 
                crop = 'wheat' AND
                irrigation = 'LIM' AND 
                stat = 'mean';
            """

df = pd.read_sql_query(emulation, conn)
df.head()

,run_id,crop,irrigation,nitrogen,stat
0,25afdf9b7d25dcb14f7ec3d082a1a326f651219160f03d...,wheat,LIM,LIM_p50,mean
1,763a876f620936df12e2b5eb013c9fd82da26eda3caf0e...,wheat,LIM,UNLIM,mean
2,c8a86be9f3f2be6ab37897b9df00a767947ac9430da8cd...,wheat,LIM,LIM_p25,mean
3,d11493e3952985b46926d9dfcdfa0b689d1866667cf77e...,wheat,LIM,LIM,mean


We need to obtain and format a list of paramter columns:

In [8]:
param_cols = [f"ct.{i}" for i in df.columns if i != 'run_id']
", ".join(param_cols)

'ct.crop, ct.irrigation, ct.nitrogen, ct.stat'

### Combining Run Output with Parameters

In [9]:
output_query = f"""
                 SELECT * from output
                 WHERE model = '{model}'
                 limit 1
              """
conn = engine.connect()
df = pd.read_sql_query(output_query, conn)

In [10]:
output_cols = [f"output.{i}" for i in df.columns]
join_cols = output_cols + param_cols
", ".join(join_cols)

'output.id, output.run_id, output.model, output.latitude, output.longitude, output.polygon, output.datetime, output.feature_name, output.feature_value, output.feature_description, output.admin1, output.admin2, output.city, output.state, output.country, ct.crop, ct.irrigation, ct.nitrogen, ct.stat'

In [11]:
join_query = f"""
                 SELECT {", ".join(join_cols)} FROM output
                 LEFT JOIN ({ct}) as ct
                 on output.run_id = ct.run_id
                 WHERE model = '{model}'
              """
conn = engine.connect()
df = pd.read_sql_query(join_query, conn)

In [12]:
df.head()

,id,run_id,model,latitude,longitude,polygon,datetime,feature_name,feature_value,feature_description,admin1,admin2,city,state,country,crop,irrigation,nitrogen,stat
0,1,d11493e3952985b46926d9dfcdfa0b689d1866667cf77e...,yield_anomalies_lpjml,65.25,25.75,None,2018-01-01,yield level,4.127357,Percent increase or decrease in yield from bas...,Oulu,Northern Ostrobothnia,None,Oulu,Finland,wheat,LIM,LIM,mean
1,2,d11493e3952985b46926d9dfcdfa0b689d1866667cf77e...,yield_anomalies_lpjml,64.75,25.25,None,2018-01-01,yield level,2.323871,Percent increase or decrease in yield from bas...,Oulu,Northern Ostrobothnia,None,Oulu,Finland,wheat,LIM,LIM,mean
2,3,d11493e3952985b46926d9dfcdfa0b689d1866667cf77e...,yield_anomalies_lpjml,64.75,25.75,None,2018-01-01,yield level,3.867433,Percent increase or decrease in yield from bas...,Oulu,Northern Ostrobothnia,None,Oulu,Finland,wheat,LIM,LIM,mean
3,4,d11493e3952985b46926d9dfcdfa0b689d1866667cf77e...,yield_anomalies_lpjml,64.75,26.25,None,2018-01-01,yield level,3.782654,Percent increase or decrease in yield from bas...,Oulu,Northern Ostrobothnia,None,Oulu,Finland,wheat,LIM,LIM,mean
4,5,d11493e3952985b46926d9dfcdfa0b689d1866667cf77e...,yield_anomalies_lpjml,64.75,26.75,None,2018-01-01,yield level,3.514873,Percent increase or decrease in yield from bas...,Oulu,Northern Ostrobothnia,None,Oulu,Finland,wheat,LIM,LIM,mean


### Generate Model Level View
This allows us to write simple queries against the joined table using the model name as the view `alias`.

In [13]:
view_query = f"CREATE VIEW {model} AS {join_query}"
conn = engine.connect()
rs = conn.execute(view_query)

### Query from View
Note that our `table` name is our `model` name; this `table` is actually a `view`.

In [14]:
view_query = f"SELECT * from {model} limit 10"
conn = engine.connect()
df = pd.read_sql_query(view_query, conn)
df.head()

,id,run_id,model,latitude,longitude,polygon,datetime,feature_name,feature_value,feature_description,admin1,admin2,city,state,country,crop,irrigation,nitrogen,stat
0,1,d11493e3952985b46926d9dfcdfa0b689d1866667cf77e...,yield_anomalies_lpjml,65.25,25.75,None,2018-01-01,yield level,4.127357,Percent increase or decrease in yield from bas...,Oulu,Northern Ostrobothnia,None,Oulu,Finland,wheat,LIM,LIM,mean
1,2,d11493e3952985b46926d9dfcdfa0b689d1866667cf77e...,yield_anomalies_lpjml,64.75,25.25,None,2018-01-01,yield level,2.323871,Percent increase or decrease in yield from bas...,Oulu,Northern Ostrobothnia,None,Oulu,Finland,wheat,LIM,LIM,mean
2,3,d11493e3952985b46926d9dfcdfa0b689d1866667cf77e...,yield_anomalies_lpjml,64.75,25.75,None,2018-01-01,yield level,3.867433,Percent increase or decrease in yield from bas...,Oulu,Northern Ostrobothnia,None,Oulu,Finland,wheat,LIM,LIM,mean
3,4,d11493e3952985b46926d9dfcdfa0b689d1866667cf77e...,yield_anomalies_lpjml,64.75,26.25,None,2018-01-01,yield level,3.782654,Percent increase or decrease in yield from bas...,Oulu,Northern Ostrobothnia,None,Oulu,Finland,wheat,LIM,LIM,mean
4,5,d11493e3952985b46926d9dfcdfa0b689d1866667cf77e...,yield_anomalies_lpjml,64.75,26.75,None,2018-01-01,yield level,3.514873,Percent increase or decrease in yield from bas...,Oulu,Northern Ostrobothnia,None,Oulu,Finland,wheat,LIM,LIM,mean
